In [4]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# 📁 Paths
BASE_DIR = "C:\\Users\\MANJU\\Desktop\\FYP_MoreData\\split_data"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

# 🔢 Parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 64
EPOCHS = 30

# 🧪 Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.3,
    shear_range=0.3,
    brightness_range=[0.6, 1.4],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# 📦 Data Loaders
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# 📏 Class Weights
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_generator.classes),
                                     y=train_generator.classes)
class_weights = dict(enumerate(class_weights))

# 🧠 Load MobileNetV2 Base
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False  # Freeze initially

# 🏗️ Custom Classifier
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)  # ⬅️ New BatchNorm layer
x = Dropout(0.4)(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# 🧪 Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0003),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 📌 Callbacks
checkpoint = ModelCheckpoint("best_mobilenetv2_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# 🚀 Initial Training (Top Layers)
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# 🔓 Fine-Tuning: Unfreeze last 30 layers
for layer in base_model.layers[-30:]:
    layer.trainable = True

# 🔁 Re-compile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 🚀 Fine-Tuning Training (Few more epochs)
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,  # fine-tune with smaller epochs
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# 🧾 Evaluate Model
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"\n✅ Final Test Accuracy after Fine-Tuning: {accuracy * 100:.2f}%")



Found 226 images belonging to 2 classes.
Found 48 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


C:\Users\MANJU\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5080 - loss: 0.9547  
Epoch 1: val_accuracy improved from -inf to 0.75000, saving model to best_mobilenetv2_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.5161 - loss: 0.9471 - val_accuracy: 0.7500 - val_loss: 0.6673 - learning_rate: 3.0000e-04
Epoch 2/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6615 - loss: 0.7218
Epoch 2: val_accuracy did not improve from 0.75000
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.6593 - loss: 0.7273 - val_accuracy: 0.7500 - val_loss: 0.5781 - learning_rate: 3.0000e-04
Epoch 3/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5753 - loss: 0.7499  
Epoch 3: val_accuracy did not improve from 0.75000
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.5815 - loss: 0.7451 - val_accuracy: 0.7500 - val_loss: 0.5248 - learning_rate: 3.0000e-04
Epoch 4/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6501 - loss: 0.5988
Epoch 4: val_accuracy did not improve from 0.75000
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.6537 - loss: 0.6102 - val_accuracy: 0.7500 - val_loss: 0.5045 - learning_rate: 3.0000e-04
Epoch 5/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 

4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7096 - loss: 0.6083 - val_accuracy: 0.7917 - val_loss: 0.4666 - learning_rate: 3.0000e-04
Epoch 7/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7266 - loss: 0.5325
Epoch 7: val_accuracy did not improve from 0.79167
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.7291 - loss: 0.5356 - val_accuracy: 0.7917 - val_loss: 0.4268 - learning_rate: 3.0000e-04
Epoch 8/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6972 - loss: 0.4539
Epoch 8: val_accuracy improved from 0.79167 to 0.83333, saving model to best_mobilenetv2_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.7020 - loss: 0.4531 - val_accuracy: 0.8333 - val_loss: 0.4057 - learning_rate: 3.0000e-04
Epoch 9/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7362 - loss: 0.5477
Epoch 9: val_accuracy did not improve from 0.83333
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.7367 - loss: 0.5413 - val_accuracy: 0.8125 - val_loss: 0.3947 - learning_rate: 3.0000e-04
Epoch 10/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7092 - loss: 0.4675
Epoch 10: val_accuracy did not improve from 0.83333
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.7063 - loss: 0.4762 - val_accuracy: 0.8125 - val_loss: 0.3932 - learning_rate: 3.0000e-04
Epoch 11/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7342 - loss: 0.5686
Epoch 11: val_accuracy did not improve from 0.83333
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.7423 - loss: 0.5608 - val_accuracy: 0.8125 - val_loss: 0.3978 - learning_rate: 3.0000e-04
Epoch 12/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 


✅ Final Test Accuracy after Fine-Tuning: 82.00%
✅ Fine-tuned & BatchNorm-enhanced MobileNetV2 model saved successfully!
